# Tarjeta de datos 4

Hipótesis 4: Las provincias situadas en el sur de España (aquellas al sur de Madrid) tienden a utilizar menos dispositivos de aprovechamiento de energías renovables que las del norte.  


### Preparación del entorno

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import os

DATA_PATH = os.path.join("..", "data/silver/")

### Carga de Datos

In [ ]:
dispositivos_df = pd.read_csv(DATA_PATH + 'dispositivos_renovable.csv', sep=';', encoding='latin')
dispositivos_df.info()

In [ ]:
renta_df = pd.read_csv(DATA_PATH + 'renta_media_hogar.csv', sep=';', encoding='latin')
renta_df.info()

In [ ]:
hogar_df = pd.read_csv(DATA_PATH + 'tipo_nucleo_familiar.csv', sep=';', encoding='latin')
hogar_df.info()

In [ ]:
edad_df = pd.read_csv(DATA_PATH + 'distribucion_edad.csv', sep=';', encoding='latin')
edad_df.info()

In [ ]:
produccion_df = pd.read_csv(DATA_PATH + 'produccion_lugar.csv', sep=';', encoding='latin')
produccion_df.info()

### Procesamiento

In [ ]:
scaler = MinMaxScaler()
renta_df['Renta normalizada'] = scaler.fit_transform(renta_df[['Total']])
renta_df = renta_df.groupby('Provincias')['Renta normalizada'].mean().reset_index()

In [ ]:
hogar_df['Número de hijos'] = hogar_df['Número de hijos'].replace({
    '0 hijos conviviendo': 0,
    '1 hijo conviviendo': 1,
    '2 o más hijos conviviendo': 2,
    '0 hijos conviviendo menores de 25 años': 0,
    '1 hijo conviviendo menor de 25 años': 1,
    '2 hijos o más conviviendo menores de 25 años': 2
})
hogar_df['Ponderación'] = hogar_df['Total'] * hogar_df['Número de hijos']
n_hijos_medio_df = hogar_df.groupby('Provincias').apply(lambda x: x['Ponderación'].sum() / x['Total'].sum()).reset_index()
n_hijos_medio_df.columns = ['Provincias', 'Número de hijos medio']

In [ ]:
edad_df['Edad (grupos quinquenales)'] = edad_df['Edad (grupos quinquenales)'].replace({
    'De 0 a 4 años': 2,
    'De 5 a 9 años': 7,
    'De 10 a 14 años  ': 12,
    'De 15 a 19 años  ': 17,
    'De 20 a 24 años': 22,
    'De 25 a 29 años': 27,
    'De 30 a 34 años': 32,
    'De 35 a 39 años': 37,
    'De 40 a 44 años': 42,
    'De 45 a 49 años': 47,
    'De 50 a 54 años': 52,
    'De 55 a 59 años': 57,
    'De 60 a 64 años': 62,
    'De 65 a 69 años': 67,
    'De 70 a 74 años  ': 72,
    'De 75 a 79 años  ': 77,
    'De 80 a 84 años  ': 82,
    'De 85 a 89 años  ': 87,
    'De 90 a 94 años  ': 92,
    'De 95 a 99 años  ': 97,
    '100 y más años': 100
})
mediana_edad_df = edad_df.groupby('Provincias').apply(lambda x: x.loc[x['Total'].cumsum() >= x['Total'].sum() / 2, 'Edad (grupos quinquenales)'].iloc[0]).reset_index()
mediana_edad_df.columns = ['Provincias', 'Mediana edad']

In [ ]:
produccion_df['Provincia'] = produccion_df['Provincia'].replace({
    'Alicante': 'Alicante/Alacant',
    'Álava': 'Araba/Álava',
    'Guipúzcoa': 'Gipuzkoa',
    'Lérida': 'Lleida',
    'La Rioja': 'Rioja, La',
    'Gerona': 'Girona',
    'Vizcaya': 'Bizkaia',
    'Castellón': 'Castellón/Castelló',
    'Valencia': 'Valencia/València',
    ' Madrid': 'Madrid',
    'A Coruña': 'La Coruña',
    'Región de Murcia': 'Murcia'
})

produccion_agrupada = produccion_df.groupby('Provincia')['Potencia kWh'].agg(['mean', 'median', lambda x: x.quantile(0.25), lambda x: x.quantile(0.75)]).reset_index()
produccion_agrupada.columns = ['Provincia', 'Producción media', 'Producción mediana', 'Producción percentil 25', 'Producción percentil 75']

todas_provincias = dispositivos_df['Provincias'].unique()
faltantes = set(todas_provincias) - set(produccion_agrupada['Provincia'])
faltantes_df = pd.DataFrame(faltantes, columns=['Provincia'])
faltantes_df['Producción media'] = 0
faltantes_df['Producción mediana'] = 0
faltantes_df['Producción percentil 25'] = 0
faltantes_df['Producción percentil 75'] = 0
produccion_agrupada = pd.concat([produccion_agrupada, faltantes_df], ignore_index=True)

In [ ]:
dispositivos_agrupados = dispositivos_df.groupby(['Provincias', 'Dispone de dispositivo de energía renovable'])['Total'].sum().unstack().fillna(0)
dispositivos_agrupados['Porcentaje con dispositivo'] = dispositivos_agrupados['Si'] / (dispositivos_agrupados['Si'] + dispositivos_agrupados['No']) * 100

In [ ]:
combined_df = pd.DataFrame({
    'Provincias': dispositivos_agrupados.index,
    'Porcentaje con dispositivo': dispositivos_agrupados['Porcentaje con dispositivo'],
    'Renta normalizada': renta_df.set_index('Provincias')['Renta normalizada'],
    'Número de hijos medio': n_hijos_medio_df.set_index('Provincias')['Número de hijos medio'],
    'Mediana edad': mediana_edad_df.set_index('Provincias')['Mediana edad'],
    'Producción media': produccion_agrupada.set_index('Provincia')['Producción media']
})

dendrogram = sch.dendrogram(sch.linkage(combined_df.drop('Provincias', axis=1), method='complete'), labels=combined_df['Provincias'].values)
plt.show()